# Binary Logisitic Regression Model Training Via sklearn

## Tech Spec
* Google Cloud Compute Engine
* n1-standard-4 (4 vCPUs, 15 GB memory)
* Debian GNU/ Linux 9

## Performance
Hyperparameter tuning over the parameter grid ----- took ----- minutes.

Traning over ------ samples took ----- minutes.

## Model Training

In [1]:
import csv
import datetime as dt
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
import time

from imblearn.under_sampling import RandomUnderSampler
from scipy.sparse import csc_matrix, hstack 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.metrics import accuracy_score, \
                            auc, \
                            confusion_matrix, \
                            log_loss, make_scorer, \
                            roc_auc_score, roc_curve, \
                            precision_recall_curve, \
                            precision_score, \
                            recall_score, \
                            f1_score
from sklearn.model_selection import GridSearchCV, \
                                    learning_curve, \
                                    StratifiedKFold, \
                                    train_test_split

### Load Data

In [2]:
data = []
with open('../data/training_data.csv') as f:
    reader = csv.reader(f, delimiter=';', quoting=csv.QUOTE_NONE)
    for row in reader:
        data.append(row)
col_names = data.pop(0)
df = pd.DataFrame(data, columns=col_names)
df.head()

del data
gc.collect()

0

### Brief Data Exploration

In [3]:
df.shape

(3738937, 17)

The training dataset has over 3.5 million records and 17 features. 

In [ ]:
numerical_feat = ['startCount', 'viewCount', 'clickCount', 'installCount', 'install', 'startCount1d', 'startCount7d']
categorical_nominal_feat = ['campaignId', 'sourceGameId', 'country', 'platform', 'softwareVersion', 'connectionType', 'deviceType']
for feat in numerical_feat:
    df[feat] = df[feat].astype('int')
df['install'] = df['install'].astype('int')
df.info()

The target variable is denoted by the binary variable _install_. The remaining features can be broken down into the categories:

**Numerical**
1. startCount
2. viewCount
3. clickCount
4. installCount
5. startCount1d
6. startCount7d

**Datetime**
1. timestamp
2. lastStart

**Categorical nominal**
1. id
2. campaignId
3. platform
4. softwareVersion
5. sourceGameId
6. country
7. connectionType
8. deviceType


In [ ]:
df['install'].value_counts()

We find that the class distribution of the install to no-install status is extremely imbalanced at 1:82. 

In [ ]:
df_num = df[numerical_feat]
df_num.corr()

We find that the install status is not linearly correlated with any of the numerical variables. Some of the features such as _startCount_ and _viewCount_ are very strongly correlated as expected.

In [ ]:
sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [7, 4]})
sns.distplot(
    df['startCount'], norm_hist=False, kde=False, bins=20, hist_kws={"range": [1, 100], "alpha": 1}
).set(xlabel='startCount', ylabel='Count');
plt.show()

The distribution of the numerical features are fat-tailed power law distributions. Hence, when it comes to scaling these features, using the standard scaling will be a bad approach. While we only show the startCount in this plot, the other numerical features look very similar.

In [ ]:
for feat in categorical_nominal_feat:
    print(feat)
    print("==========")
    print(df[feat].value_counts())
    print("        ")

This shows us that the cardinality of the campaignId and sourceGameId features are very high.

### Feature Engineering: Use timestamp and lastStart to create timeSinceLastStart

In [ ]:
def datetime_parser(datetime_str):
    try:
        date_str, time_str = datetime_str.split("T")
    except:
        return 
    time_str=time_str[:8]
    year, month, day = date_str.split("-")
    hour, minut, sec = time_str.split(":")
    return dt.datetime(int(year), int(month), int(day), int(hour), int(minut), int(sec))
        

def time_diff_in_minutes(dt_0, dt_1):
    return np.round((dt_1 - dt_0).total_seconds() / 60.0, 0)

In [ ]:
df.timestamp = df.timestamp.apply(datetime_parser)
df.lastStart = df.lastStart.apply(datetime_parser)

df['timeSinceLastStart'] = df.apply(lambda row: time_diff_in_minutes(row['lastStart'], row['timestamp']), axis=1).fillna(0) 
df = df.drop(['timestamp', 'lastStart', 'id'], axis=1)

 ### OHE Categorical Nominal Features 

In [ ]:
X_num = df[['startCount', 'viewCount', 'clickCount', 'installCount', 'startCount1d', 'startCount7d', 'timeSinceLastStart']].values
X_cat = df[['campaignId', 'sourceGameId', 'country', 'platform', 'softwareVersion', 'connectionType', 'deviceType']]

y = df['install']
enc = OneHotEncoder(handle_unknown='ignore')
X_cat = enc.fit_transform(X_cat)
X = hstack((X_cat, X_num))

# del df
# gc.collect()

The feature vectors here are a combination of sparse and dense features (though the columns are overwhelmingly sparse).

### Dataset Training/Test Split

Our goal is to find the optimal hyperparameters for the logistic regression model. In order to do that the training dataset is further split into training and test datasets. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

X_train = csc_matrix(X_train)
y_train = y_train.values
X_test = csc_matrix(X_test)
y_test = y_test.values

One of the two simple options to balance the datset is undersampling which would involve matching the number of non-install classes to be the same as the number of install classes in the dataset.

In [ ]:
def undersample_fit(x, y):
    zero_indices = []
    one_indices = []
    for idx, class_label in enumerate(y):
        if class_label == 0:
            zero_indices.append(idx)
        else:
            one_indices.append(idx)
    
    resampled_indices = one_indices + random.sample(zero_indices, len(one_indices))
    random.shuffle(resampled_indices)
    resampled_x, resampled_y = [], []
    for idx in resampled_indices:
        resampled_x.append(x[idx].toarray()[0])
        resampled_y.append(y[idx])
    return csc_matrix(resampled_x), resampled_y

## 2.3 Performance Metrics

The metrics used to measure the classifier performance other than AUROC, log-loss and prediction bias are the precision and recall.

In [ ]:
def log_loss_score(clf, x, y):
    return log_loss(y, clf.predict_proba(x))

def auroc_score(clf, x, y):
    return roc_auc_score(y, clf.predict_proba(x)[:, 1])

## 2.4 Hyperparameter Tuning Via Stratified Cross Validated Grid Search

In [ ]:
def imbalanced_cross_validation_score(clf, x, y, cv, scoring):
    train_scores = []
    test_scores = []
    skf = StratifiedKFold(n_splits=cv, shuffle=False)
    
    for train_idx, test_idx in skf.split(x,y):
        xfold_train_sampled, yfold_train_sampled = undersample_fit(x[train_idx], y[train_idx])
        clf.fit(xfold_train_sampled, yfold_train_sampled)
        
        train_score = scoring(clf, xfold_train_sampled, yfold_train_sampled)
        test_score  = scoring(clf, x[test_idx], y[test_idx])
        
        train_scores.append(train_score)
        test_scores.append(test_score)
        
    print("Train score: {} +/- {}, CV score: {} +/- {}".format(np.round(np.mean(train_scores), 2),
                                                               np.round(np.std(train_scores), 2),
                                                               np.round(np.mean(test_scores), 2),
                                                               np.round(np.std(test_scores), 2))) 

The hyperparameter grid is specified by the C values and the penalty metrics below. 

In [ ]:
C = [0.01, 0.1, 1.0, 10.0, 100.0]
penalty = ['l1', 'l2']
for c in C:
    for pen in penalty:
        print("C: {}, Penalty: {}".format(str(c), pen))
        print("====================")
        t_0 = time.time()
        pipe_lr = Pipeline([('scl', RobustScaler(with_centering=False)),
                           ('clf', LogisticRegression(C=c, penalty=pen, solver='liblinear', random_state=0))])
        imbalanced_cross_validation_score(pipe_lr, X_train, y_train, 3, auroc_score)
        print("  ")

Using the stratified cross validation schema along with undersampling the non-install class to balance the dataset, we find that C=0.1 and penalty='l1' gives the optimal log-loss score of 0.63 +/- 0 on the test set and 0.62 +/- 0 on the training set. Meanwhile, for the AUROC we find a test score of 0.72 +/- 0.01 and train score of 0.73 +/- 0.01.

## 2.5 Learning Curve For Optimal Classifier

In [ ]:
pipe_lr = Pipeline([('scl', RobustScaler(with_centering=False)),
                    ('clf', LogisticRegression(C=0.1, penalty='l1', solver='liblinear', random_state=0))])
imbalanced_cross_validation_score(pipe_lr, X_train, y_train, 3, log_loss_score)

I ran the preceding code using various sample sizes and compared their performances.
* Sample size: 100,000, CV log-loss score - train: 0.62, test: 0.64
* Sample size: 200,000, CV log-loss score - train: 0.61, test: 0.63
* Sample size: 400,000, CV log-loss score - train: 0.61, test: 0.62
* Sample size: 800,000, CV log-loss score - train: 0.62, test: 0.62
* Sample size: 160,0000, CV log-loss score - train: 0.63, cv: 0.64

Based on these results, it seems that using smaller subsample of the training data is a viable option to get around memory errors.

## 2.6 Other Performance Metrics

In [ ]:
x, y = undersample_fit(X_train, y_train)
pipe_lr = Pipeline([('scl', RobustScaler(with_centering=False)),
                    ('clf', LogisticRegression(C=0.1, penalty='l1', solver='liblinear', random_state=0))])
pipe_lr.fit(x, y)
y_pred = pipe_lr.predict(X_test)
print("Precision: {}%".format(int(100 * precision_score(y_test, y_pred))))
print("Recall: {}%".format(int(100 * recall_score(y_test, y_pred))))
print("Log-loss: {}%".format(int(100 * log_loss_score(pipe_lr, X_test, y_test))))
print("AUROC: {}%".format(int(100 * roc_auc_score(y_test, pipe_lr.predict_proba(X_test)[:, 1]))))
tn, fp, fn, tp = confusion_matrix(y_pred, y_test).ravel()
print("True Negatives: {}, Fale Positives: {}, False Negatives: {}, True Positives: {}".format(tn, fp, fn, tp))
print("Prediction bias: {}".format(sum(y_pred) / len(y_pred) - sum(y_test) / len(y_test)))

We find that the model has a very low precision of 4% but perhaps this is jutified by the recall of 72%. 